<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

# Kubeflow Pipelines with Katib component

In this notebook you will:
- Create Katib Experiment using random algorithm.
- Use median stopping rule as an early stopping algorithm.
- Use Kubernetes Job with mxnet mnist training container as a Trial template.
- Create Pipeline to get the optimal hyperparameters.

Reference documentation:
- https://kubeflow.org/docs/components/katib/experiment/#random-search
- https://kubeflow.org/docs/components/katib/early-stopping/
- https://kubeflow.org/docs/pipelines/overview/concepts/component/

**Note**: This Pipeline runs in the multi-user mode. Follow [this guide](https://github.com/kubeflow/katib/tree/master/examples/v1beta1/kubeflow-pipelines#multi-user-pipelines-setup) to give your Notebook access to Kubeflow Pipelines.

In [1]:
# Install required packages (Kubeflow Pipelines and Katib SDK).
!pip install kfp==2.3.0
!pip install kubeflow-katib==0.17.0
!pip install kubernetes==27.2.0
!pip install protobuf==3.20.*

  Using cached kubernetes-31.0.0-py2.py3-none-any.whl (1.9 MB)


  Using cached protobuf-4.25.5-cp37-abi3-manylinux2014_x86_64.whl (294 kB)


ERROR: kfp 2.3.0 has requirement kubernetes<27,>=8.0.0, but you'll have kubernetes 31.0.0 which is incompatible.
ERROR: kfp 2.3.0 has requirement protobuf<4,>=3.13.0, but you'll have protobuf 4.25.5 which is incompatible.
ERROR: kfp-pipeline-spec 0.2.2 has requirement protobuf<4,>=3.13.0, but you'll have protobuf 4.25.5 which is incompatible.
ERROR: google-api-core 2.19.2 has requirement google-auth<3.0.dev0,>=2.14.1, but you'll have google-auth 1.35.0 which is incompatible.
  Attempting uninstall: kubernetes
    Found existing installation: kubernetes 26.1.0


    Uninstalling kubernetes-26.1.0:
      Successfully uninstalled kubernetes-26.1.0


  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:


...

     |████████████████████████████████| 1.5 MB 332 kB/s 


ERROR: kfp 2.3.0 has requirement kubernetes<27,>=8.0.0, but you'll have kubernetes 27.2.0 which is incompatible.
ERROR: kfp 2.3.0 has requirement protobuf<4,>=3.13.0, but you'll have protobuf 4.25.5 which is incompatible.
  Attempting uninstall: kubernetes
    Found existing installation: kubernetes 31.0.0


    Uninstalling kubernetes-31.0.0:
      Successfully uninstalled kubernetes-31.0.0


  Using cached protobuf-3.20.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)


ERROR: kubeflow-katib 0.17.0 has requirement protobuf<5,>=4.21.12, but you'll have protobuf 3.20.3 which is incompatible.
ERROR: kfp 2.3.0 has requirement kubernetes<27,>=8.0.0, but you'll have kubernetes 27.2.0 which is incompatible.
ERROR: google-api-core 2.19.2 has requirement google-auth<3.0.dev0,>=2.14.1, but you'll have google-auth 1.35.0 which is incompatible.
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5


In [2]:
import kfp
import kfp.dsl as dsl
from kfp import components

from kubeflow.katib import KatibClient, ApiClient, V1beta1Experiment
from kubeflow.katib.models import (
    V1beta1ExperimentSpec,
    V1beta1AlgorithmSpec,
    V1beta1ObjectiveSpec,
    V1beta1ParameterSpec,
    V1beta1FeasibleSpace,
    V1beta1TrialTemplate,
    V1beta1TrialParameterSpec,
    V1beta1EarlyStoppingSpec,
    V1beta1EarlyStoppingSetting,
)

## Define an Experiment

You have to create an Experiment object before deploying it. This Experiment is similar to [this](https://github.com/kubeflow/katib/blob/master/examples/v1beta1/early-stopping/median-stop.yaml) YAML.

In [3]:
# Experiment name and namespace.
experiment_name = "median-stop"
experiment_namespace = "kubeflow-user-example-com"

# Trial count specification.
max_trial_count = 18
max_failed_trial_count = 3
parallel_trial_count = 2

# Objective specification.
objective=V1beta1ObjectiveSpec(
    type="minimize",
    goal= 0.001,
    objective_metric_name="loss",
)

# Algorithm specification.
algorithm=V1beta1AlgorithmSpec(
    algorithm_name="random",
)

# Early Stopping specification.
early_stopping=V1beta1EarlyStoppingSpec(
    algorithm_name="medianstop",
    algorithm_settings=[
        V1beta1EarlyStoppingSetting(
            name="min_trials_required",
            value="2"
        )
    ]
)

# Experiment search space.
# In this example we tune learning rate, number of layer and optimizer.
# Learning rate has bad feasible space to show more early stopped Trials.
parameters=[
    V1beta1ParameterSpec(
        name="lr",
        parameter_type="double",
        feasible_space=V1beta1FeasibleSpace(
            min="0.01",
            max="0.3"
        ),
    ),
    V1beta1ParameterSpec(
        name="momentum",
        parameter_type="double",
        feasible_space=V1beta1FeasibleSpace(
            min="0.5",
            max="0.9"
        ),
    ),
]


## Define a Trial template

In this example, the Trial's Worker is the Kubernetes Job.

In [4]:
# JSON template specification for the Trial's Worker Kubernetes Job.
trial_spec={
    "apiVersion": "batch/v1",
    "kind": "Job",
    "spec": {
        "template": {
            "metadata": {
                "annotations": {
                     "sidecar.istio.io/inject": "false"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "training-container",
                        "image": "docker.io/kubeflowkatib/pytorch-mnist-cpu:v0.14.0",
                        "command": [
                            "python3",
                            "/opt/pytorch-mnist/mnist.py",
                            "--epochs=1",
                            "--batch-size=16",
                            "--lr=${trialParameters.learningRate}",
                            "--momentum=${trialParameters.momentum}",
                        ]
                    }
                ],
                "restartPolicy": "Never"
            }
        }
    }
}

# Configure parameters for the Trial template.
# We set the retain parameter to "True" to not clean-up the Trial Job's Kubernetes Pods.
trial_template=V1beta1TrialTemplate(
    retain=True,
    primary_container_name="training-container",
    trial_parameters=[
        V1beta1TrialParameterSpec(
            name="learningRate",
            description="Learning rate for the training model",
            reference="lr"
        ),
        V1beta1TrialParameterSpec(
            name="momentum",
            description="Momentum for the training model",
            reference="momentum"
        ),
    ],
    trial_spec=trial_spec
)

## Define an Experiment specification

Create an Experiment specification from the above parameters.

In [5]:
experiment_spec=V1beta1ExperimentSpec(
    max_trial_count=max_trial_count,
    max_failed_trial_count=max_failed_trial_count,
    parallel_trial_count=parallel_trial_count,
    objective=objective,
    algorithm=algorithm,
    early_stopping=early_stopping,
    parameters=parameters,
    trial_template=trial_template
)

# Create a Pipeline using Katib component

The best hyperparameters are printed after Experiment is finished.
The Experiment is not deleted after the Pipeline is finished.

In [6]:
# # Get the Katib launcher.
# katib_experiment_launcher_op = components.load_component_from_url(
#     "https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/katib-launcher/component.yaml")
#
# @dsl.pipeline(
#     name="Launch Katib early stopping Experiment",
#     description="An example to launch Katib Experiment with early stopping"
# )
#
# def median_stop():
#
#     # Katib launcher component.
#     # Experiment Spec should be serialized to a valid Kubernetes object.
#     op = katib_experiment_launcher_op(
#         experiment_name=experiment_name,
#         experiment_namespace=experiment_namespace,
#         experiment_spec=ApiClient().sanitize_for_serialization(experiment_spec),
#         experiment_timeout_minutes=60,
#         delete_finished_experiment=False)
#
#     # Output container to print the results.
#     op_out = dsl.ContainerOp(
#         name="best-hp",
#         image="library/bash:4.4.23",
#         command=["sh", "-c"],
#         arguments=["echo Best HyperParameters: %s" % op.output],
#     )

# Create pipeline component to launch Katib experiment.
@dsl.component
def launch_katib_experiment():
    katib_client = KatibClient()
    experiment = katib_client.create_experiment(
        V1beta1Experiment(
            api_version="kubeflow.org/v1beta1",
            kind="Experiment",
            metadata={"name": experiment_name, "namespace": experiment_namespace},
            spec=experiment_spec
        )
    )
    print(f"Experiment '{experiment_name}' created in namespace '{experiment_namespace}'.")
    is_created = katib_client.is_experiment_created(experiment_name, experiment_namespace)
    is_running = katib_client.is_experiment_running(experiment_name, experiment_namespace)
    print("-----------------------------------------------------------------------------")
    print(f"Is experiment created: {is_created}\n Is experiment running: {is_running}")
    return experiment_name

#remove later
from pprint import pprint

@dsl.component
def get_best_hyperparameters(experiment_name: str):
    katib_client = KatibClient()
    best_trial = katib_client.get_best_trial(name=experiment_name, namespace=experiment_namespace)
    print("Best trial's hyperparameters:", best_trial)

# Define Kubeflow pipeline.
@dsl.pipeline(
    name="Launch Katib early stopping Experiment Pipeline",
    description="A pipeline to launch Katib Experiment with early stopping"
)
def katib_pipeline():
    launch_task = launch_katib_experiment()
    # print("Launch Task Details:")
    # pprint(vars(launch_task))
    # get_best_hp_task = get_best_hyperparameters(launch_task.output)

# Run the Kubeflow Pipeline

You can check the Katib Experiment info in the Katib UI.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
# Run the Kubeflow Pipeline in the user's namespace.
kfp.Client().create_run_from_pipeline_func(katib_pipeline,  namespace=experiment_namespace, arguments={})

/home/kerido/.local/lib/python3.8/site-packages/kfp/client/client.py:158: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


ERROR:root:Failed to get healthz info attempt 1 of 5.
Traceback (most recent call last):
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp/client/client.py", line 435, in get_kfp_healthz
    return self._healthz_api.get_healthz()
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api/healthz_service_api.py", line 63, in get_healthz
    return self.get_healthz_with_http_info(**kwargs)  # noqa: E501
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api/healthz_service_api.py", line 134, in get_healthz_with_http_info
    return self.api_client.call_api(
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 364, in call_api
    return self.__call_api(resource_path, method,
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 188, in __call_api
    raise e
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 181, in __call_api

ERROR:root:Failed to get healthz info attempt 2 of 5.
Traceback (most recent call last):
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp/client/client.py", line 435, in get_kfp_healthz
    return self._healthz_api.get_healthz()
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api/healthz_service_api.py", line 63, in get_healthz
    return self.get_healthz_with_http_info(**kwargs)  # noqa: E501
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api/healthz_service_api.py", line 134, in get_healthz_with_http_info
    return self.api_client.call_api(
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 364, in call_api
    return self.__call_api(resource_path, method,
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 188, in __call_api
    raise e
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 181, in __call_api

ERROR:root:Failed to get healthz info attempt 3 of 5.
Traceback (most recent call last):
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp/client/client.py", line 435, in get_kfp_healthz
    return self._healthz_api.get_healthz()
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api/healthz_service_api.py", line 63, in get_healthz
    return self.get_healthz_with_http_info(**kwargs)  # noqa: E501
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api/healthz_service_api.py", line 134, in get_healthz_with_http_info
    return self.api_client.call_api(
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 364, in call_api
    return self.__call_api(resource_path, method,
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 188, in __call_api
    raise e
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 181, in __call_api

ERROR:root:Failed to get healthz info attempt 4 of 5.
Traceback (most recent call last):
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp/client/client.py", line 435, in get_kfp_healthz
    return self._healthz_api.get_healthz()
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api/healthz_service_api.py", line 63, in get_healthz
    return self.get_healthz_with_http_info(**kwargs)  # noqa: E501
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api/healthz_service_api.py", line 134, in get_healthz_with_http_info
    return self.api_client.call_api(
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 364, in call_api
    return self.__call_api(resource_path, method,
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 188, in __call_api
    raise e
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 181, in __call_api

ERROR:root:Failed to get healthz info attempt 5 of 5.
Traceback (most recent call last):
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp/client/client.py", line 435, in get_kfp_healthz
    return self._healthz_api.get_healthz()
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api/healthz_service_api.py", line 63, in get_healthz
    return self.get_healthz_with_http_info(**kwargs)  # noqa: E501
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api/healthz_service_api.py", line 134, in get_healthz_with_http_info
    return self.api_client.call_api(
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 364, in call_api
    return self.__call_api(resource_path, method,
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 188, in __call_api
    raise e
  File "/home/kerido/.local/lib/python3.8/site-packages/kfp_server_api/api_client.py", line 181, in __call_api

TimeoutError: Failed getting healthz endpoint after 5 attempts.